<h1 style="text-align:center">New York City Taxi Trip Duration</h1>

<h3>Load Libraries</h3>

In [ ]:
import numpy as np
import pandas as pd

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns

<h3>Load Dataset</h3>

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
print("Train set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

<h3>Data exploration</h3>

In [ ]:
# Observation of the first 5 elements in the training set
train_df.head()

In [ ]:
# Check the types or each features
train_df.dtypes

Analyze missing data 

In [ ]:
print(train_df.isnull().sum(), "\n")
print(test_df.isnull().sum())

There's no missing data in both the train and test dataframe

<u>Target Feature</u>

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
train_df['trip_duration'].describe()

Let's observe the skewness of the trip duration

In [ ]:
train_df['trip_duration'].skew()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

sns.distplot(train_df['trip_duration'])